# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

De tabel INHABITANTS\_PER\_MUNICIPALITY wordt direct in het VWSStatic schema ingelezen. Hiervoor wordt een tabel aangemaakt in het VWSStage schema alsemede een stored procedure om de data van Stage naar Static te kopieren en gedurende de kopieer-actie aan te passen aan de gewenste gemeentelijke indeling. 

Daarnaast moet er nog een Process worden toegevoegd aan de workflow.

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

DROP TABLE IF EXISTS [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY]
GO

CREATE TABLE [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1),
	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE(),
	[REGION] [VARCHAR](50) NOT NULL,
	[GM_CODE] [VARCHAR](50) NOT NULL,
	[VR_CODE] [VARCHAR](50) NOT NULL,
	[VR_NAME] [VARCHAR](50) NOT NULL,
	[INHABITANTS] [VARCHAR](50) NOT NULL
) ON [PRIMARY];
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] (
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE(),
    [REGIO] [VARCHAR](50) NOT NULL,
	[GMCODE] [VARCHAR](50) NOT NULL,
	[VRCODE] [VARCHAR](50) NOT NULL,
	[VRNAME] [VARCHAR](50) NOT NULL,
	[INHABITANTS] [INT] NOT NULL
);
GO

# **STORED PROCEDURES**

---

De tabel INHABITANTS\_PER\_MUNICIPALITY wordt direct in het VWSStatic schema ingelezen. Hiervoor wordt een tabel aangemaakt in het VWSStage schema alsemede een stored procedure om de data van Stage naar Static te kopieren en gedurende de kopieer-actie aan te passen aan de gewenste gemeentelijke indeling. 

Daarnaast moet er nog een Process worden toegevoegd aan de workflow.

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC]
AS
BEGIN

    INSERT INTO [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] (
            [REGIO], 
            [GMCODE], 
            [VRCODE], 
            [VRNAME], 
            [INHABITANTS], 
            [DATE_LAST_INSERTED]
        )
    SELECT  ISNULL(B.[GM_NAME_NEW],A.[REGION]),
            ISNULL(B.[GM_CODE_NEW],A.[GM_CODE]),
            ISNULL(B.[VR_CODE],A.[VR_CODE]),
            ISNULL(B.[VR_NAME],A.[VR_NAME]),
            CAST(ROUND(SUM(CAST(A.[INHABITANTS] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT),
            MAX(A.[DATE_LAST_INSERTED])
    FROM [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY] A
    LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE]=1
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[GM_CODE] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY])    
    GROUP BY ISNULL(B.[GM_NAME_NEW],A.[REGION]),
            ISNULL(B.[GM_CODE_NEW],A.[GM_CODE]),
            ISNULL(B.[VR_CODE],A.[VR_CODE]),
            ISNULL(B.[VR_NAME],A.[VR_NAME])
END;
GO